In [1]:
import numpy as np
from time import time
np.random.seed(123)

# import Sequntial that is a linear stack of neural network layers (use for feed forward)

In [2]:
from keras.models import Sequential

Using TensorFlow backend.


In [3]:
from keras.layers import Dense, Dropout, Activation, Flatten

In [4]:
from keras.layers import Convolution2D, MaxPooling2D

In [5]:
from keras.utils import np_utils

In [6]:
from keras.callbacks import TensorBoard

In [7]:
from keras.datasets import mnist

In [8]:
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

In [9]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 15s 1us/step


In [10]:
print(np.shape(X_train))
Image.fromarray(X_train[0])

(60000, 28, 28)


In [11]:
print(y_train[0])

5


# need to reshape our data to decompose the X data as a single array

In [12]:
def array_add_channel(a):
    '''
    Take an array of images and reshapes the 2D image to hae an added channel single array
    
    Parameters:
    -----------
    a : 3d numpy array; 
        first dimesion is the number of images
        second dimesion is the rows
        third dimension is the cols
        NOTE: tensorflow likes data to be in the order (Number, Height, Width, Channel)
    '''
    return(a.reshape(a.shape[0], a.shape[1], a.shape[2], 1))

def pre_process_x(a):
    '''
    Takes a raw array of images, add_channels the data, converts to float, and normalizes
    '''
    a = array_add_channel(a)
    return(a.astype('float32')/255)

In [13]:
X_train = pre_process_x(X_train)
X_test = pre_process_x(X_test)
np.shape(X_train)

(60000, 28, 28, 1)

In [14]:
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

In [15]:
print(Y_train.shape)

(60000, 10)


In [16]:
print(X_train.shape)

(60000, 28, 28, 1)


# Now we define the architecture of the model, given the data is now formatted

In [17]:
model = None
model = Sequential() # not a recurrent neural net..

In [18]:
filters = 32
kernel = (3,3)
input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])
model.add(Convolution2D(filters, kernel, activation='relu', input_shape=input_shape))

Parameters of Convolution2D
    
    1) number of filters
    2) kernel size
    3) strides (default (1,1))

In [19]:
print(model.output_shape)

(None, 26, 26, 32)


In [20]:
model.add(Convolution2D(filters, kernel, activation='relu'))

In [21]:
print(model.output_shape) 

(None, 24, 24, 32)


When we add a convolution to a first layer, we are applying the kernel to filters too?

In [22]:
model.add(MaxPooling2D(pool_size=(2,2)))  # resample and aggregate using a 2x2 window (should be a )

In [23]:
print(model.output_shape)

(None, 12, 12, 32)


In [24]:
model.add(Dropout(0.25))

In [25]:
print(model.output_shape)

(None, 12, 12, 32)


In [26]:
model.add(Flatten())

In [27]:
print(model.output_shape)

(None, 4608)


In [28]:
12*12*32

4608

In [29]:
model.add(Dense(128, activation='relu'))

In [30]:
print(model.output_shape)

(None, 128)


In [31]:
model.add(Dropout(0.5))
model.add(Dense(Y_train.shape[1], activation='softmax'))

In [32]:
print(model.output_shape)

(None, 10)


# Now the model is fully constructed. We can set up the same thing again in a single function and compile it!

In [33]:
def twolayermodel(filters, kernel, input_shape):
    model = Sequential()
    model.add(Convolution2D(filters, kernel, activation='relu', input_shape=input_shape))
    model.add(Convolution2D(filters, kernel, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    return(model)

In [34]:
simple_model = twolayermodel(filters, kernel, input_shape)

In [35]:
simple_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [36]:
tensorboard = TensorBoard(log_dir='logs/{}'.format(time()))

In [37]:
simple_model.fit(X_train, Y_train, batch_size=32, epochs=10, verbose=1, callbacks=[tensorboard])

/usr/local/lib/python3.5/dist-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/10
60000/60000 [==============================] - 10s 167us/step - loss: 0.2128 - acc: 0.9350
Epoch 2/10
60000/60000 [==============================] - 9s 148us/step - loss: 0.0906 - acc: 0.9734
Epoch 3/10
60000/60000 [==============================] - 9s 143us/step - loss: 0.0694 - acc: 0.9784
Epoch 4/10
60000/60000 [==============================] - 9s 144us/step - loss: 0.0594 - acc: 0.9820
Epoch 5/10
60000/60000 [==============================] - 9s 143us/step - loss: 0.0506 - acc: 0.9847
Epoch 6/10
60000/60000 [==============================] - 9s 143us/step - loss: 0.0457 - acc: 0.9857
Epoch 7/10
60000/60000 [==============================] - 9s 149us/step - loss: 0.0384 - acc: 0.9882
Epoch 8/10
60000/60000 [==============================] - 9s 143us/step - loss: 0.0354 - acc: 0.9882
Epoch 9/10
60000/60000 [==============================] - 9s 144us/step - loss: 0.0340 - acc: 0.9889
Epoch 10/10
60000/60000 [==============================] - 9s 143us/step - loss: 0.0311 - 

In [38]:
score = simple_model.evaluate(X_test, Y_test, verbose=0)

In [39]:
score

[0.035811764282543575, 0.99099999999999999]